# Analisis de informacion geografica

A partir de un conjunto de datos que abarca diversas operaciones mineras alrededor del mundo, titulado ``'Mineral Ores Around The World Cleaned.csv'``, se busca realizar el análisis de:

* La ubicación de las mineras.
* La frecuencia relativa de los minerales extraídos por las operaciones mineras.
* La identificación de las operaciones mineras más cercanas de distinto tipo.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import folium

In [ ]:
#lectura de datos
df_minas = pd.read_csv("./Mineral Ores Around The World Cleaned.csv")
df_minas.shape

(3125, 19)

In [ ]:
df_minas.columns

Index(['site_name', 'latitude', 'longitude', 'region', 'country', 'state',
       'county', 'com_type', 'commod1', 'commod2', 'commod3', 'oper_type',
       'prod_size', 'dev_stat', 'ore', 'gangue', 'work_type', 'hrock_type',
       'arock_type'],
      dtype='object')

In [ ]:
df_minas.oper_type.value_counts()

Underground            1523
Surface                1135
Surface-Underground     357
Placer                   79
Well                     16
Processing Plant          6
Brine Operation           4
Leach                     3
Offshore                  2
Name: oper_type, dtype: int64

## Mapa de centros de explotacion

In [ ]:
gdf_minas = gpd.GeoDataFrame(
    df_minas,
    geometry=gpd.points_from_xy(df_minas.longitude, df_minas.latitude, crs = "EPSG:4326")
)
gdf_minas.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
categorias = list(gdf_minas.oper_type.unique()); categorias
paleta_colores = sns.color_palette("husl", n_colors=len(categorias))
colores_hex = paleta_colores.as_hex()
asignacion_colores_hex = dict(zip(categorias, colores_hex)); asignacion_colores_hex

# creamos mapa
f_map = folium.Map(location=[40.00, -110.000], zoom_start=4,
                   tiles="cartodbpositron",
                   width=900, height=600,
                   prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''

    color = asignacion_colores_hex.get(point['oper_type'], 'gray')
    text = "País: {} <br> Tipo: {}".format(point.country, point.oper_type)
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=4, fill = True, fill_opacity=0.8,
                        stroke=False,
                        color = color,
                        tooltip=text).add_to(f_map)
gdf_minas.apply(plotDot, axis = 1)

# creamos leyenda
legend_html = '''
     <div style="position: fixed;
                 bottom: 20px; left: 50px; width: 725px; height: 40px;
                 border: 2px solid grey; z-index: 9999; font-size: 10px;
                 background-color: white; display: flex; align-items: center;
                 ">&nbsp; <b style="margin-right: 10px;">Leyenda</b>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Surface-Underground
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Surface
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Underground
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Placer
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Well
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Processing Plant
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Brine Operation
                   </div>
                   <div style="display: flex; align-items: center; margin-right: 10px;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Offshore
                   </div>
                   <div style="display: flex; align-items: center;">
                       <i class="fa fa-circle fa-1x" style="color:{}; margin-right: 5px;"></i> Leach
                   </div>
      </div>
     '''.format(*list(asignacion_colores_hex.values()))
f_map.get_root().html.add_child(folium.Element(legend_html))
f_map

El mapa muestra un conjunto de operaciones mineras en distintos continentes.

Esto se logró transformando las coordenadas de latitud y longitud de nuestro dataset, a una geometría de puntos, bajo un sistema de coordenadas geográficas EPSG:4326 (WGS84). Es importante destacar que las coordenadas de nuestro conjunto de datos interno ya venían en longitud y latitud, por lo que no se aplicó ninguna operación adicional para transformarlas a geometría de puntos. Por último, mediante las funciones de Folium, se logró crear el mapa observado.

## Frecuencia relativa de minerales

In [ ]:
gdf_minas_explode = (gdf_minas[["country", "oper_type", "commod1"]]
                     .assign(commod1 = lambda x: x.commod1.str.split(", ")).explode("commod1") )

In [ ]:
tbl_frelativa = (gdf_minas_explode.commod1.value_counts()
                 .to_frame()
                 .reset_index()
                 .rename(columns={"index" : "Mineral", "commod1" : "Cantidad"})
                 .assign(Total = lambda x : x.Cantidad.sum(),
                         Frecuencia_relativa = lambda x :round(x.Cantidad/ x.Total,3)*100 ))
tbl_frelativa.head(10)

,Mineral,Cantidad,Total,Frecuencia_relativa
0,Copper,543,4094,13.3
1,Gold,531,4094,13.0
2,Silver,361,4094,8.8
3,Lead,312,4094,7.6
4,Zinc,311,4094,7.6
5,Iron,250,4094,6.1
6,Phosphorus-Phosphates,184,4094,4.5
7,Aluminum,144,4094,3.5
8,Tungsten,136,4094,3.3
9,Nickel,125,4094,3.1


La tabla superior muestra el top 10 de los minerales que más extraen las distintas operaciones mineras de nuestro conjunto de datos. Se observa que los principales minerales extraídos son Cobre con un 13.3%, Oro con un 13%, y Plata con un 8.8%

## Frecuencia relativa de los minerales por tipo de operacion


In [ ]:
gdf_minas_group = (gdf_minas_explode.groupby(["oper_type", "commod1"],as_index= False).agg("size")
                   .rename(columns={"size": "cantidad"})
                   .sort_values(by = ["oper_type"]))

In [ ]:
gdf_minas_group['total_por_operacion'] = gdf_minas_group.groupby("oper_type")['cantidad'].transform('sum')
gdf_minas_group["frelativa"] = (gdf_minas_group.cantidad / gdf_minas_group.total_por_operacion).round(3) *100
gdf_minas_group = gdf_minas_group.sort_values(by = ["oper_type", "frelativa"], ascending = False)
gdf_minas_group["frelativa_acum"]  = gdf_minas_group.groupby("oper_type")["frelativa"].transform(lambda x : x.cumsum())

gdf_minas_group_top10  = gdf_minas_group.groupby("oper_type", as_index = False).apply(lambda x : x.head(10))
gdf_minas_group_top10 = gdf_minas_group_top10.reset_index().drop(["level_0", "level_1"], axis = 1)
gdf_minas_group_top10

,oper_type,commod1,cantidad,total_por_operacion,frelativa,frelativa_acum
0,Brine Operation,Magnesite,4,4,100.0,100.0
1,Leach,Copper,3,3,100.0,100.0
2,Offshore,Barium-Barite,1,2,50.0,50.0
3,Offshore,Sulfur,1,2,50.0,100.0
4,Placer,Gold,67,106,63.2,63.2
5,Placer,Tin,7,106,6.6,69.8
6,Placer,Titanium,5,106,4.7,74.5
7,Placer,Silver,5,106,4.7,79.2
8,Placer,REE,5,106,4.7,83.9
9,Placer,Zirconium,3,106,2.8,86.7


Parecido al caso anterior, la tabla superior muestra los top 10 minerales más extraídos por tipo de operación minera.

Se observa que hay tipos de operaciones que se dedican exclusivamente a un tipo de mineral, por ejemplo, la operación "Brine Operation" solo extrae Magnesio. Por otro lado, tenemos operaciones que manejan más minerales, como son "Surface", "Surface-Underground" y "Underground". Además, se logra ver que dentro del top 10 de tipos de minerales que manejan las operaciones "Surface", solo están cubriendo el 67% del total de su diversidad de minerales extraídos.

In [ ]:
chart = alt.Chart(gdf_minas_group_top10).mark_bar().encode(
    y=alt.X('oper_type:N', axis=alt.Axis(title='Tipo de operacion',labelAngle = -45)),
    x=alt.Y('frelativa:Q', axis=alt.Axis(title='(%) del mineral en la operación')),
    color=alt.Color('commod1', title='Minerales'),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'frelativa',
      sort='descending'
    ),
    tooltip=['commod1:N']
).properties(
    width=600,
    height=400,
    title='Minerales extraidos por tipo de operaciónn minera'
)
chart


alt.Chart(...)

Como respaldo a lo anterior, la gráfica busca mostrar un orden de magnitud de la proporción de tipos de minerales que maneja cada operación.

Es importante destacar que la paleta de colores no puede utilizar un color único para cada uno de los minerales, por lo que hay minerales con el mismo color. Para abordar esto, se dejó la gráfica con el tooltip activado.

## Operacion minera mas cercana

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
gdf_proyectado = gdf_minas.to_crs(epsg=3395)

In [ ]:
# Obtener las coordenadas como una matriz
coordenadas_proyectadas = gdf_proyectado['geometry'].apply(lambda geom: (geom.x, geom.y)).to_list()
coordenadas_proyectadas = np.array(coordenadas_proyectadas)

# Calcular la matriz de distancias
matriz_distancias = cdist(coordenadas_proyectadas, coordenadas_proyectadas, metric='euclidean')

# Convertir la matriz de distancias
df_distancias = pd.DataFrame(matriz_distancias, index=gdf_proyectado.index, columns=gdf_proyectado.index)
df_distancias = df_distancias/1000

In [ ]:
df_distancias

,0,1,2,3,4,5,6,7,8,9,...,3115,3116,3117,3118,3119,3120,3121,3122,3123,3124
0,0.000000,50.628938,8326.589985,8675.793415,9629.531172,6846.066082,9029.515634,8072.843922,8003.485978,8277.725067,...,10068.285463,10024.189576,7809.814488,8272.644374,9924.531219,7870.825878,8086.120469,9632.631764,10021.514141,10037.455193
1,50.628938,0.000000,8376.914807,8725.928630,9679.006965,6896.445635,9078.835668,8122.433896,8053.044543,8328.289793,...,10118.751766,10074.723087,7860.027842,8323.000270,9974.937201,7919.480532,8136.621648,9682.080416,10071.929556,10087.879358
2,8326.589985,8376.914807,0.000000,433.293358,1603.377387,1482.577598,1244.592441,807.802507,852.269683,496.065360,...,1762.678649,1754.304885,538.184141,72.157005,1604.442009,1456.421100,398.410979,1619.193419,1702.782866,1720.411154
3,8675.793415,8725.928630,433.293358,0.000000,1170.307777,1856.172744,857.940595,803.066886,870.984186,857.485852,...,1500.432384,1534.716153,871.177646,505.278057,1319.823340,1414.680606,824.975374,1186.350422,1418.390327,1439.304648
4,9629.531172,9679.006965,1603.377387,1170.307777,0.000000,2935.055320,614.289400,1559.611587,1627.470508,1990.763702,...,1389.707710,1551.153151,1965.760793,1675.457502,1211.545167,1851.989868,1993.559523,24.447505,1262.950777,1286.640050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,7870.825878,7919.480532,1456.421100,1414.680606,1851.989868,1678.862457,1240.494562,649.836291,605.342089,1900.856330,...,2830.199433,2902.309941,1195.580038,1480.760628,2634.364731,0.000000,1684.423113,1848.224121,2726.475759,2750.252101
3121,8086.120469,8136.621648,398.410979,824.975374,1993.559523,1257.906327,1640.552650,1068.136853,1091.234305,256.015003,...,1983.864092,1940.042184,532.921945,327.676962,1849.764207,1684.423113,0.000000,2010.175760,1944.452732,1958.763594
3122,9632.631764,9682.080416,1619.193419,1186.350422,24.447505,2944.576869,612.743733,1564.220886,1631.641807,2009.533577,...,1412.446006,1574.482325,1977.021527,1691.306590,1234.840465,1848.224121,2010.175760,0.000000,1285.688626,1309.323114
3123,10021.514141,10071.929556,1702.782866,1418.390327,1262.950777,3176.064832,1632.555721,2191.211420,2263.501520,1784.435421,...,126.757890,306.124853,2235.177152,1752.344838,99.045774,2726.475759,1944.452732,1285.688626,0.000000,24.909748


In [ ]:
#Dataframe con solo el tipo de operacion y su geometria
df_oper_type = pd.DataFrame(gdf_proyectado[['oper_type', "geometry"]], index=gdf_proyectado.index)

In [ ]:
for index in df_distancias.index:
    # Obtener las distancias del índice actual
    distancias = df_distancias.loc[index].copy()

    # Encontrar el índice del centro de operación minera más cercano con un tipo de operación diferente
    distancias[ df_oper_type['oper_type'] == df_oper_type.loc[index, 'oper_type'] ] = 1e15
    indice_mas_cercano = np.argmin(distancias)
    distancia_mas_cercano = np.min(distancias)

    # indice_mas_cercano = np.argmin( distancias[ df_oper_type['oper_type'] != df_oper_type.loc[index, 'oper_type'] ] )
    # distancia_mas_cercano = np.min( distancias[ df_oper_type['oper_type'] != df_oper_type.loc[index, 'oper_type'] ] )

    # Obtener el tipo de operación del índice más cercano y asignarlo al índice actual
    tipo_operacion_mas_cercano = df_oper_type.iloc[indice_mas_cercano, 0]
    geometry_mas_cercano = df_oper_type.iloc[indice_mas_cercano, 1]

    gdf_proyectado.at[index, 'tipo_operacion_mas_cercano'] = tipo_operacion_mas_cercano
    gdf_proyectado.at[index, 'distancia_al_mas_cercano'] = round(distancia_mas_cercano, 2)
    gdf_proyectado.at[index, 'geom_del_mas_cercano'] = geometry_mas_cercano
    gdf_proyectado.at[index, 'X_del_mas_cercano'] = geometry_mas_cercano.x
    gdf_proyectado.at[index, 'Y_del_mas_cercano'] = geometry_mas_cercano.y


In [ ]:
#Ojo que la columna de punto agregada no esta en formato geometria, es string.
gdf_proyectado.columns

Index(['site_name', 'latitude', 'longitude', 'region', 'country', 'state',
       'county', 'com_type', 'commod1', 'commod2', 'commod3', 'oper_type',
       'prod_size', 'dev_stat', 'ore', 'gangue', 'work_type', 'hrock_type',
       'arock_type', 'geometry', 'tipo_operacion_mas_cercano',
       'distancia_al_mas_cercano', 'geom_del_mas_cercano', 'X_del_mas_cercano',
       'Y_del_mas_cercano'],
      dtype='object')

In [ ]:
gdf_proyectado.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   site_name                   3125 non-null   object  
 1   latitude                    3125 non-null   float64 
 2   longitude                   3125 non-null   float64 
 3   region                      1065 non-null   object  
 4   country                     3125 non-null   object  
 5   state                       2630 non-null   object  
 6   county                      1779 non-null   object  
 7   com_type                    3112 non-null   object  
 8   commod1                     3125 non-null   object  
 9   commod2                     967 non-null    object  
 10  commod3                     1228 non-null   object  
 11  oper_type                   3125 non-null   object  
 12  prod_size                   1023 non-null   object  
 13  dev_stat  

In [ ]:
gdf_proyectado[["site_name", "country", "oper_type", "geometry", "tipo_operacion_mas_cercano", "distancia_al_mas_cercano", "geom_del_mas_cercano" ]]

,site_name,country,oper_type,geometry,tipo_operacion_mas_cercano,distancia_al_mas_cercano,geom_del_mas_cercano
0,Red Dog mine,United States,Surface-Underground,POINT (-18126152.686 10428145.870),Underground,669.65,POINT (-17496539.664696153 10200071.557433754)
1,Su-Lik,United States,Surface-Underground,POINT (-18167897.495 10456793.039),Placer,715.10,POINT (-18490083.931144647 9818383.459068298)
2,Many Values Pegmatite Prospect,United States,Surface,POINT (-11819341.369 4991581.533),Underground,12.05,POINT (-11825352.621504119 5002029.032142979)
3,Democrat Hill deposit,United States,Underground,POINT (-11728776.284 4567858.629),Surface-Underground,2.29,POINT (-11730076.495723972 4569739.560997341)
4,Adobe Walls Mine,United States,Underground,POINT (-11527380.401 3415009.987),Surface-Underground,10.44,POINT (-11516976.4813035 3414195.8200544165)
...,...,...,...,...,...,...,...
3120,McIlroy property,United States,Underground,POINT (-13126463.735 4349247.089),Surface-Underground,127.01,POINT (-13012265.522629976 4404828.62861283)
3121,Nichols Ranch ISL,United States,Well,POINT (-11801721.720 5389602.708),Underground,108.94,POINT (-11746903.549952284 5295462.510995752)
3122,Tarrant Property in Sec. 39,United States,Underground,POINT (-11542940.639 3396153.702),Surface-Underground,1.17,POINT (-11544017.098812094 3395695.1305260346)
3123,Pike City Prospect,United States,Surface,POINT (-10419726.977 4021762.455),Surface-Underground,7.56,POINT (-10419627.902885187 4029321.240270492)


Para cada una de las operaciones mineras de nuestro conjunto de datos, se encontró la operación minera de distinto tipo más cercana. La tabla superior muestra el resultado.

Se presenta la distancia en kilómetros entre las operaciones mineras en la columna ``distancia_al_mas_cercano``. Cabe destacar que, para calcular la distancia entre las distintas operaciones en kilómetros, fue necesario proyectar los datos a un sistema de coordenadas proyectado EPSG:3395.